In [6]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn 
import re
import nltk 
from nltk.corpus import stopwords
from textblob import Word


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **Feature Engineering**

In [8]:
from google.colab import drive
drive.mount('/content/drive')
Data_Essay_01 = pd.read_csv("/content/drive/MyDrive/IntelliTech-DataSet/EssaySet01.csv")
Data_Essay_01.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score
0,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0
2,3,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4.0,3.0,7.0
3,4,"Dear Local Newspaper, @CAPS1 I have found that...",5.0,5.0,10.0
4,5,"Dear @LOCATION1, I know having computers has a...",4.0,4.0,8.0


## Counting Spelling Mistakes

In [9]:
def removePunctuations(sentence):
  """
    Removes punctuations from text

    Args:
      sentence: Essay of each student
    
    Returns: 
      String

  """
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  newSentence = ""
  for word in sentence:
      if (word in punctuations):
          newSentence = newSentence + " "
      else: 
          newSentence = newSentence + word
  return newSentence

def SplitWords(sentence):
  """
    Creates tokens of sentence

    Args:
      sentence: Essay of each student
    
    Returns: 
      String[]
      
  """
  return sentence.split()

def check_word_spelling(word):
  """
    Checks spelling of each word

    Args:
      word: Words (Tokens) of each essay 
    
    Returns: 
      int
      
  """
  word = Word(word)
  
  result = word.spellcheck()

  # result [0][0] contains the bool value if the spelling is correct or not
  # result [0][1] contains the confidence for the suggest correct spelling

  if word != result[0][0]:
    # print(f'Spelling of "{word}" is not correct!')
    # print(f'Correct spelling of "{word}": "{result[0][0]}" (with {result[0][1]} confidence).')
    return 1
  return 0

In [10]:
# df_1 = data.query('essay_set == 1')
# cols = ["Essay" , "Number of Spelling Mistakes"]
# Mistakes = []
# for index, row in df_1.iterrows():
#   sentence = row["essay"]
#   PreProcessed0 = removePunctuations(sentence)
#   PreProcessed1 = PreProcessed0.lower()
#   Words = SplitWords(PreProcessed1)
#   count = 0 
#   for word in Words:
#     count = count + check_word_spelling(word)
#   Mistakes.append(count)

# Featured_df = pd.DataFrame(list(zip(df_1 , Mistakes)) , columns = cols)
# Featured_df.head()

In [16]:
# Counting Sentences per Essay
def sentence_count(essay):

    sentence_no = nltk.sent_tokenize(essay)
    return len(sentence_no)
  

Data_Essay_01['Sent_Count'] = Data_Essay_01['Essay'].apply(sentence_count)
Data_Essay_01.sample()


,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Sent_Count
733,736,"Dear @CAPS1, @CAPS2 I understand your one tryi...",5.0,5.0,10.0,25


In [19]:
# Counting Words per Essay
def word_count(essay):
  
  cleaned_essay = re.sub('[^a-zA-Z]',' ',essay) 
  word_no = nltk.word_tokenize(cleaned_essay)
  return len(word_no)
 
Data_Essay_01['Word_Count'] = Data_Essay_01['Essay'].apply(word_count)
Data_Essay_01.sample()



,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Sent_Count,Word_Count
105,106,"Dear Newspaper @CAPS1, I firmly believe that c...",5.0,6.0,11.0,30,369


In [ ]:
# Counting Characters per Essay
def 